### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-11"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_SNAPSHOT]

# drop the one that lifetime is less than 4 seconds
# df_vehicle = df_vehicle.loc[(df_vehicle["lifetime"] > 4),]

# create a timestamp_unix at the end of lifespan
# df_vehicle["timestamp_unix_end"] = df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"].astype(np.int64))
# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

## Cross vs Vehicle

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

         camera       timestamp_precise    xmid    ymid  timestamp_unix
2730  mbk-14-11 2024-04-28 02:00:18.401  1911.5   408.5      1714269618
2713  mbk-14-11 2024-04-28 02:28:18.469  1217.0   743.5      1714271298
2700  mbk-14-11 2024-04-28 02:30:09.642  1573.0   475.0      1714271409
2694  mbk-14-11 2024-04-28 02:35:01.000   621.0   980.5      1714271701
2675  mbk-14-11 2024-04-28 02:56:10.105  1585.0   498.5      1714272970
2676  mbk-14-11 2024-04-28 02:57:19.086  2019.0   362.5      1714273039
2659  mbk-14-11 2024-04-28 03:02:30.324  1616.0   448.0      1714273350
2660  mbk-14-11 2024-04-28 03:02:37.482  1691.0   518.0      1714273357
2662  mbk-14-11 2024-04-28 03:02:48.319  1874.0   524.0      1714273368
2661  mbk-14-11 2024-04-28 03:02:49.319  1630.5   525.5      1714273369
2664  mbk-14-11 2024-04-28 03:02:54.477  1602.0   457.5      1714273374
2663  mbk-14-11 2024-04-28 03:03:02.714  1621.0   490.5      1714273382
2666  mbk-14-11 2024-04-28 03:03:08.713  1614.0   499.5      171

## Reverse vs Vehicle

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

## TOGETHER


In [4]:
df_vehicle["lifetime"]

2340    1.480
2341    1.000
2336    1.001
2337    1.360
2331    0.000
        ...  
53      1.000
54      1.518
39      1.479
40      0.760
19      1.240
Name: lifetime, Length: 252, dtype: float64

In [5]:
df_vehicle = match_reverse_to_vehicle(df_reverse, df_vehicle)
df_vehicle = match_cross_to_vehicle(
    df_cross,
    df_vehicle,
)


df_vehicle = df_vehicle[
    (df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714288020)
]
df_vehicle = df_vehicle[
    [
        *conf.VEHICLE_INTEREST_SNAPSHOT,
        "cross_count",
        "reverse_count",
        "original_video_name",
        # "frame_no",
    ]
]

df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)
print(
    df_vehicle[
        [
            "camera",
            "timestamp_precise",
            "lifetime",
            "cross_count",
            "reverse_count",
            "vehicle_type",
            "plate_number",
            "timestamp_unix",
            "timestamp_unix_end",
            "xmid",
            "ymid",
        ]
    ].to_string()
)

                 camera       timestamp_precise  lifetime  cross_count  reverse_count vehicle_type plate_number  timestamp_unix  timestamp_unix_end    xmid   ymid
1867  mbk-14-11-vehicle 2024-04-28 06:02:29.843    18.875            2              0        sedan        41360      1714284149          1714284167  1388.5  120.5
1860  mbk-14-11-vehicle 2024-04-28 06:05:35.235    28.751            1              1        sedan      6กม9678      1714284335          1714284363  1406.5  126.0
1861  mbk-14-11-vehicle 2024-04-28 06:06:08.106    15.516            4              0          suv       มฎ5222      1714284368          1714284383  1403.0  140.5
1862  mbk-14-11-vehicle 2024-04-28 06:06:28.620    32.111            2              0    hatchback      3กฐ7375      1714284388          1714284420  1535.0  187.0
1863  mbk-14-11-vehicle 2024-04-28 06:08:58.459    51.865            0              0       tuktuk        สก573      1714284538          1714284589  1892.5  214.0
1850  mbk-14-11-vehicl

## TODO 
- check cross walk angle?
- allow customer able to be classified to the next car (not found in the current time stamp)?
- check displacement?